In [1]:
%run ../Requirements.ipynb

24/06/20 18:35:04 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
24/06/20 18:35:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 18:35:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='ronitgupta28',
    database='MusicFusion'
)
cursor = conn.cursor()

Music Industry News

In [3]:
api_key = News_API

url = f'https://newsapi.org/v2/everything?q=Music Industry&apiKey={api_key}'

response = requests.get(url)
data = response.json()

In [4]:
if response.status_code == 200 and data['status'] == 'ok':
    articles = data['articles']
    df = pd.DataFrame(articles)
else:
    print(f"Error: {response.status_code}, {data['message']}")
    
df = df.drop(['source', 'content'], axis=1)
df = df.dropna(subset=['urlToImage'])

df.to_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/GeneralMusicNews')

In [5]:
columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])

drop_table_query = "DROP TABLE IF EXISTS GeneralMusicNews;"
cursor.execute(drop_table_query)

create_table_query = f"CREATE TABLE IF NOT EXISTS GeneralMusicNews ({columns});"
cursor.execute(create_table_query)

insert_query = f"INSERT INTO GeneralMusicNews ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"

delete_query = f"DELETE FROM GeneralMusicNews WHERE author is NULL;"


for index, row in df.iterrows():
    values = tuple([None if pd.isna(val) else val for val in row])
    cursor.execute(insert_query, values)
    
cursor.execute(delete_query)
conn.commit()

Music Concerts News

In [6]:
api_key = News_API
url = f'https://newsapi.org/v2/everything?apiKey={api_key}&q=Latest Music Concerts'
response = requests.get(url)
data = response.json()

In [7]:
if response.status_code == 200 and data['status'] == 'ok':
    articles = data['articles']
    df = pd.DataFrame(articles)
else:
    print(f"Error: {response.status_code}, {data['message']}")
    
df = df.drop(['source', 'content'], axis=1)
df = df.dropna(subset=['urlToImage'])

df.to_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/MusicConcertsNews')

In [8]:
df = pd.read_parquet('/Users/ronitguptaaa/Documents/Music-Fusion-APP/Mart/MusicConcertsNews')

In [9]:
columns = ", ".join([f"`{col}` VARCHAR(1024)" for col in df.columns])

drop_table_query = "DROP TABLE IF EXISTS MusicConcertsNews;"
cursor.execute(drop_table_query)

create_table_query = f"CREATE TABLE IF NOT EXISTS MusicConcertsNews ({columns});"
cursor.execute(create_table_query)

insert_query = f"INSERT INTO MusicConcertsNews ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"

delete_query = f"DELETE FROM MusicConcertsNews WHERE author is NULL;"


for index, row in df.iterrows():
    values = tuple([None if pd.isna(val) else val for val in row])
    cursor.execute(insert_query, values)
    
cursor.execute(delete_query)
conn.commit()